In [17]:
from shapely.geometry import Point, mapping, shape
from shapely.ops import transform
from pyproj import CRS, Transformer
import json

def create_circle_geojson_shapely(lon, lat, radius_meters, output_file):
    # Determine UTM zone for projection
    utm_zone = int((lon + 180) / 6) + 1
    is_northern = lat >= 0

    crs_wgs84 = CRS("EPSG:4326")
    crs_utm = CRS.from_proj4(
        f"+proj=utm +zone={utm_zone} +datum=WGS84 +units=m +{'north' if is_northern else 'south'}"
    )

    # Create transformer functions
    to_utm = Transformer.from_crs(crs_wgs84, crs_utm, always_xy=True).transform
    to_wgs84 = Transformer.from_crs(crs_utm, crs_wgs84, always_xy=True).transform

    # Create buffer around point in UTM (meter units)
    center_utm = transform(to_utm, Point(lon, lat))
    circle_utm = center_utm.buffer(radius_meters, resolution=64)  # resolution = smoothness

    # Reproject back to WGS84 for GeoJSON
    circle_wgs84 = transform(to_wgs84, circle_utm)

    # Create GeoJSON Feature
    feature = {
        "type": "Feature",
        "geometry": mapping(circle_wgs84),
        "properties": {
            "center": [lon, lat],
            "radius_m": radius_meters
        }
    }

    feature_collection = {
        "type": "FeatureCollection",
        "features": [feature]
    }

    # Save to file
    with open(output_file, 'w') as f:
        json.dump(feature_collection, f, indent=2)

    print(f"Circle saved to {output_file}")


In [6]:
#create 1 circle test
create_circle_geojson_shapely(
    lon=-82.400011,
    lat=34.848497,
    radius_meters=1609.34,  # 1 mile
    output_file=r'C:\Users\Joe\Desktop\geojsons\circle.geojson'
)


Circle saved to C:\Users\Joe\Desktop\geojsons\circle.geojson


In [18]:
#create 50 circles for 50 mile radius
for i in range(50):
    create_circle_geojson_shapely(
        lon=-82.400011,
        lat=34.848497,
        radius_meters=(1609.34*(i+1)),  # 1 mile
        output_file=rf'C:\Users\Joe\Desktop\geojsons\circle{i+1}.geojson'
    )


Circle saved to C:\Users\Joe\Desktop\geojsons\circle1.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle2.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle3.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle4.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle5.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle6.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle7.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle8.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle9.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle10.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle11.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle12.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle13.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle14.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle15.geojson
Circle saved to C:\Users\Joe\Desktop\geojsons\circle16.geojson
C

In [1]:
import json
from shapely.geometry import shape
from shapely.ops import transform
from pyproj import CRS, Transformer


def calculate_geojson_area(geojson_file):
    # Load the GeoJSON
    with open(geojson_file, 'r') as f:
        data = json.load(f)

    total_area_m2 = 0.0

    for feature in data['features']:
        geom = shape(feature['geometry'])

        # Get centroid for local projection (UTM)
        centroid = geom.centroid
        lon, lat = centroid.x, centroid.y

        # Determine UTM zone
        utm_zone = int((lon + 180) / 6) + 1
        is_northern = lat >= 0

        crs_wgs84 = CRS("EPSG:4326")
        crs_utm = CRS.from_proj4(
            f"+proj=utm +zone={utm_zone} +datum=WGS84 +units=m +{'north' if is_northern else 'south'}"
        )

        # Create projection function
        project = Transformer.from_crs(crs_wgs84, crs_utm, always_xy=True).transform

        # Project geometry to meters (UTM) and calculate area
        geom_projected = transform(project, geom)
        total_area_m2 += geom_projected.area

        return total_area_m2  # square meters


In [2]:
print(calculate_geojson_area(r"C:\Users\Joe\Desktop\geojsons\circle1.geojson"))

8135830.288069832


In [4]:
import json
import random
from shapely.geometry import shape, Point


def get_random_point_within_geojson(geojson_file):
    # Load the GeoJSON
    with open(geojson_file, 'r') as f:
        data = json.load(f)

    # Combine all geometries
    combined_geom = shape(data['features'][0]['geometry'])
    for feature in data['features'][1:]:
        combined_geom = combined_geom.union(shape(feature['geometry']))

    # Bounding box for sampling
    minx, miny, maxx, maxy = combined_geom.bounds

    # Try generating a point inside the polygon
    for _ in range(10000):
        x = random.uniform(minx, maxx)
        y = random.uniform(miny, maxy)
        point = Point(x, y)
        if combined_geom.contains(point):
            return [point.x, point.y]  # [lon, lat]

    raise ValueError("Couldn't find a point inside the GeoJSON geometry.")


In [9]:
for _ in range(10):
    print(get_random_point_within_geojson(r"C:\Users\Joe\Desktop\geojsons\circle1.geojson"))

[-82.4102181914804, 34.8413921803846]
[-82.40388272992743, 34.83444544423445]
[-82.40445137876064, 34.84195587639627]
[-82.41003455566435, 34.85119018838175]
[-82.40057584597407, 34.85861851297995]
[-82.38298516231359, 34.849493376511376]
[-82.40553141222728, 34.857123098494355]
[-82.39673593179802, 34.84613358392156]
[-82.39021448170088, 34.85785641421446]
[-82.40562568447449, 34.84486351144768]


In [28]:
#remove area from geojson using google data, create new file
from shapely.geometry import shape, Point, mapping
from shapely.ops import transform
from pyproj import CRS, Transformer
import json

def remove_overlap_with_circle(input_geojson, lon, lat, radius_meters, output_geojson):
    # Determine UTM zone for projection
    utm_zone = int((lon + 180) / 6) + 1
    is_northern = lat >= 0

    crs_wgs84 = CRS("EPSG:4326")
    crs_utm = CRS.from_proj4(
        f"+proj=utm +zone={utm_zone} +datum=WGS84 +units=m +{'north' if is_northern else 'south'}"
    )

    # Define transformation functions
    to_utm = Transformer.from_crs(crs_wgs84, crs_utm, always_xy=True).transform
    to_wgs84 = Transformer.from_crs(crs_utm, crs_wgs84, always_xy=True).transform

    # Create the circle in UTM
    center_utm = transform(to_utm, Point(lon, lat))
    circle_utm = center_utm.buffer(radius_meters, resolution=64)

    # Load the input GeoJSON
    with open(input_geojson, 'r') as f:
        data = json.load(f)

    new_features = []
    for feature in data.get("features", []):
        geom = shape(feature["geometry"])
        geom_utm = transform(to_utm, geom)

        # Subtract the circle if it overlaps
        if geom_utm.intersects(circle_utm):
            difference = geom_utm.difference(circle_utm)
            if not difference.is_empty:
                new_geom = transform(to_wgs84, difference)
                new_feature = {
                    "type": "Feature",
                    "geometry": mapping(new_geom),
                    "properties": feature.get("properties", {})
                }
                new_features.append(new_feature)
        else:
            # No intersection, keep original
            new_features.append(feature)

    # Save updated GeoJSON
    updated_geojson = {
        "type": "FeatureCollection",
        "features": new_features
    }

    with open(output_geojson, 'w') as f:
        json.dump(updated_geojson, f, indent=2)

    print(f"Output saved to {output_geojson}")


In [29]:
remove_overlap_with_circle(r"C:\Users\Joe\Desktop\geojsons\circle1.geojson", -82.38284, 34.85111, 100, r"C:\Users\Joe\Desktop\geojsons\circle1_revised.geojson")

Output saved to C:\Users\Joe\Desktop\geojsons\circle1_revised.geojson


In [14]:
#remove area from geojson using google data, overwrite file
from shapely.geometry import shape, Point, mapping
from shapely.ops import transform
from pyproj import CRS, Transformer
import json

def remove_overlap_with_circle(input_geojson, lon, lat, radius_meters):
    # Determine UTM zone
    utm_zone = int((lon + 180) / 6) + 1
    is_northern = lat >= 0

    crs_wgs84 = CRS("EPSG:4326")
    crs_utm = CRS.from_proj4(
        f"+proj=utm +zone={utm_zone} +datum=WGS84 +units=m +{'north' if is_northern else 'south'}"
    )

    # Set up projection transformers
    to_utm = Transformer.from_crs(crs_wgs84, crs_utm, always_xy=True).transform
    to_wgs84 = Transformer.from_crs(crs_utm, crs_wgs84, always_xy=True).transform

    # Create circle in UTM
    center_utm = transform(to_utm, Point(lon, lat))
    circle_utm = center_utm.buffer(radius_meters, resolution=64)

    # Load input GeoJSON
    with open(input_geojson, 'r') as f:
        data = json.load(f)

    updated_features = []
    for feature in data.get("features", []):
        geom = shape(feature["geometry"])
        geom_utm = transform(to_utm, geom)

        # Remove overlap if present
        if geom_utm.intersects(circle_utm):
            diff = geom_utm.difference(circle_utm)
            if not diff.is_empty:
                new_geom = transform(to_wgs84, diff)
                updated_features.append({
                    "type": "Feature",
                    "geometry": mapping(new_geom),
                    "properties": feature.get("properties", {})
                })
        else:
            updated_features.append(feature)

    # Overwrite the input file
    updated_geojson = {
        "type": "FeatureCollection",
        "features": updated_features
    }

    with open(input_geojson, 'w') as f:
        json.dump(updated_geojson, f, indent=2)

    print(f"Overlapping areas removed and {input_geojson} has been updated.")


In [19]:
#cuts out iner radius to create 1 mile radius circles, updates original geojson
from shapely.geometry import shape, mapping
from shapely.ops import unary_union
import json

def remove_overlap_with_geojson(input_geojson, remove_geojson):
    # Load both GeoJSON files
    with open(input_geojson, 'r') as f:
        input_data = json.load(f)

    with open(remove_geojson, 'r') as f:
        circle_data = json.load(f)

    # Combine all geometries from the circle_geojson into a single unioned shape
    mask_shapes = [shape(f["geometry"]) for f in circle_data.get("features", [])]
    mask_union = unary_union(mask_shapes)

    updated_features = []
    for feature in input_data.get("features", []):
        geom = shape(feature["geometry"])

        # Subtract the mask if it intersects
        if geom.intersects(mask_union):
            diff = geom.difference(mask_union)
            if not diff.is_empty:
                updated_features.append({
                    "type": "Feature",
                    "geometry": mapping(diff),
                    "properties": feature.get("properties", {})
                })
        else:
            # No overlap, keep the original
            updated_features.append(feature)

    # Overwrite the input GeoJSON with updated features
    updated_geojson = {
        "type": "FeatureCollection",
        "features": updated_features
    }

    with open(input_geojson, 'w') as f:
        json.dump(updated_geojson, f, indent=2)

    print(f"Overlapping areas removed from {input_geojson} using mask from {remove_geojson}.")


In [20]:
for i in range(49):
    remove_overlap_with_geojson(rf"C:\Users\Joe\Desktop\geojsons\circle{50-i}.geojson", rf"C:\Users\Joe\Desktop\geojsons\circle{49-i}.geojson")

Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle50.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle49.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle49.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle48.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle48.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle47.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle47.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle46.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle46.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle45.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle45.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle44.geojson.
Overlapping areas removed from C:\Users\Joe\Desktop\geojsons\circle44.geojson using mask from C:\Users\Joe\Desktop\geojsons\circle43.geojson.
Overla

In [10]:
#cuts out iner radius to create 1 mile radius circles, creates new file
import os
import json
from shapely.geometry import shape, mapping
from shapely.ops import unary_union

def remove_overlap_with_geojson(input_geojson, remove_geojson):
    # Load input and removal GeoJSON files
    with open(input_geojson, 'r') as f:
        input_data = json.load(f)

    with open(remove_geojson, 'r') as f:
        remove_data = json.load(f)

    # Union of all removal geometries
    mask_shapes = [shape(f["geometry"]) for f in remove_data.get("features", [])]
    mask_union = unary_union(mask_shapes)

    updated_features = []
    for feature in input_data.get("features", []):
        geom = shape(feature["geometry"])

        if geom.intersects(mask_union):
            diff = geom.difference(mask_union)
            if not diff.is_empty:
                updated_features.append({
                    "type": "Feature",
                    "geometry": mapping(diff),
                    "properties": feature.get("properties", {})
                })
        else:
            updated_features.append(feature)

    # Create new filename by appending "_center_removed" before extension
    base, ext = os.path.splitext(input_geojson)
    output_geojson = f"{base}_center_removed{ext}"

    with open(output_geojson, 'w') as f:
        json.dump({
            "type": "FeatureCollection",
            "features": updated_features
        }, f, indent=2)

    print(f"Output written to {output_geojson} (overlapping areas removed).")


In [12]:
remove_overlap_with_geojson(r"C:\Users\Joe\Desktop\geojsons\circle2.geojson", r"C:\Users\Joe\Desktop\geojsons\circle1.geojson")

Output written to C:\Users\Joe\Desktop\geojsons\circle2_center_removed.geojson (overlapping areas removed).
